In [1]:
import pandas as pd
import numpy as np
from uncertainties import ufloat
import jinja2

In [62]:
a0, a1, a2, a3 = ufloat(1.919260427189138e+01,	3.026770213127952e+00), ufloat(5.047652979997769e+01,	2.056753680262795e+00), ufloat(2.849949852042239e+00,	3.953297870136209e-01), ufloat(-2.132518936336041e-01,	2.244482073418326e-02)
rap = ufloat(16.21818181,0.052)
rbp = ufloat(22.3,0.052)
rcp = ufloat(27,0.052)
data = {
    'I [A]': [5.08, 6.03, 6.54, 7.25, 7.6, 7.77, 8.02, 8.62, 9.09, 9.6, 10.02]
}

n = 1.4560
d = 3*10**-3 # 3mm in m
lda = 643.847e-9 # 643.847 nm in m
h = 6.62607015e-34 # Planck's constant in J*s
c = 2.998e8 # Speed of light in m/s
df = pd.DataFrame(data)

#function = a0 + a1 * x + a2 * x ^ 2 + a3 * x ^ 3
def func(x, a0, a1, a2, a3):
    return a0 + a1 * x + a2 * x ** 2 + a3 * x ** 3

df['B [mT]'] = df['I [A]'].apply(lambda x: (func(x, a0, a1, a2, a3)))
df['B, uncertainty [mT]'] = df['I [A]'].apply(lambda x: round(func(x, a0, a1, a2, a3).std_dev, 2))


df2 = pd.read_csv('data.csv')
#df2 = df2.drop(columns=df2.columns[:3])
df = pd.concat([df.reset_index(drop=True), df2.reset_index(drop=True)], axis=1)
df["valueAB"] = (df["r a p+1"]**2 - rap**2)/(df["r b p"]**2 - rap**2)
df["valueBC"] = (df["r b p+1"]**2 - rbp**2)/(rcp**2 - rbp**2)

df ["coeff"] =  (2.273861*10**-23/(df["B [mT]"]/1000))
df["muBab"] = df["valueAB"] * df["coeff"] * 10**24
df["muBbc"] = df["valueBC"] * df["coeff"] * 10**24

df["lda2 [nm]"] =-1/(1/(df["valueAB"]*2*n*d) - 1/lda)
df["R"] = (df["r a p+1"]**2 - rap**2)/(df["r b p"]**2 - rap**2)
df["ldai"] =(1/lda +df["R"]/(2*n*d))**-1 
df["delta E"] = h*c*(1/lda - 1/df["ldai"])

#df["delta e [eV]"] = (-h*c/lda  h*c/(df["lda2 [nm]"]))

df["mu B"] = df["delta E"]/(df["B [mT]"])*1000

df2 = df.drop(columns=["B, uncertainty [mT]", "valueAB", "valueBC", "coeff", "Unnamed: 8", "r a p", "r b p", "r c p"])
print(df2)
#convert to tex
print(df2.to_latex(index=False))
df2.to_latex("table.tex", index=False)
lst = list()
for mu in df["muBab"]:
    print(mu)
    lst.append(mu)

#sum everything in muBab
sum = df["muBbc"].sum()/11
print("sum mu AB:", sum)

muavg = np.mean(lst)
print("avg mu AB:", muavg)


    I [A]           B [mT]  Unnamed: 0  Unnamed: 1  Unnamed: 2  r a p+1  \
0    5.08         321+/-15        5.08      321.20          15     17.3   
1    6.03         380+/-20        6.03      380.44          20     17.5   
2    6.54         412+/-23        6.54      411.55          23     17.5   
3    7.25         454+/-27        7.25      453.68          27     17.7   
4    7.60         474+/-30        7.60      473.81          30     17.8   
5    7.77         483+/-31        7.77      483.42          31     17.8   
6    8.02         497+/-33        8.02      497.32          33     17.8   
7    8.62  (5.3+/-0.4)e+02        8.62      529.48          37     17.9   
8    9.09  (5.5+/-0.4)e+02        9.09      553.34          41     18.0   
9    9.60  (5.8+/-0.5)e+02        9.60      577.75          46     18.0   
10  10.02  (6.0+/-0.5)e+02       10.02      596.57          50     18.1   

    r b p+1       muBab       muBbc                  lda2 [nm]              R  \
0      23.0  10.8+

In [65]:
#formula
val1 = ufloat(9.9916126901952e-24 , 4.6457798405959e-25  )
val2 = ufloat( 1.0276778710790e-23 , 4.3967938958612e-25)
mub = (val1+val2)/2*10**24
print("muB:", mub)

muB: 10.13+/-0.32
